# SYCL Buffer and Accesors

The cell below writes the `compute.cpp` file containing a SYCL program that performs a parallel multiplication operation on a buffer using SYCL. The program demonstrates how to submit tasks to a queue, use accessors to manage memory, and execute kernels on the selected device.

In [ ]:
%%writefile compute.cpp
#include <sycl/sycl.hpp>
#include <vector>
#include <iostream>

int main() {

    sycl::queue myQueue;                         // The queue, Q, is the object that
                                                 // submits the task to a device.
    int const N = 10;
    sycl::buffer<int> bufA{ sycl::range{N} };    // The buffer, A, is the memory used to
                                                 // transfer data between host and device.
    const int p = 4;
    std::cout << "Running on "
                  << myQueue.get_device().get_info<sycl::info::device::name>()
                  << "\n";

    myQueue.submit([&](sycl::handler& cgh) {      // The handler, h, is the object that contains
                                                  // the single_task function to be used.

        sycl::accessor accA(bufA, cgh);          // The accessor, A_acc, is the object that
                                                  // efficiently accesses the buffer elements.

            cgh.parallel_for(N, [=](auto i) {
                accA[i] = i * p; 
                });

        });

    sycl::host_accessor result(bufA);              // host_accessor is the object that allows
                                                   // the host to access the buffer memory.

    for (int i = 0; i < N ; i++)                   // Print output
        std::cout << result[i] << " "; std::cout << "\n";
    return 0;

}

The next cell writes a shell script `run-sycl.sh`. The script sets up the SYCL environment, compiles the `compute.cpp` program using the DPC++ compiler (`icpx`), and runs the executable if the compilation is successful.

In [ ]:
%%writefile ./run-sycl.sh
#!/bin/bash
source /opt/intel/oneapi/setvars.sh > /dev/null 2>&1
icpx -fsycl compute.cpp
if [ $? -eq 0 ]; then ./a.out; fi

The cell will make the shell script executable and then runs it, which compiles and executes the SYCL program.

In [ ]:
!chmod u+x ./run-sycl.sh &&./run-sycl.sh

## Example 2:

This next example again creates a `compute.cpp` file, which demonstrates using SYCL to perform parallel computation on a GPU. The program creates a vector, uses a buffer to manage memory between the host and the device, and performs a simple multiplication operation using `parallel_for`. It highlights how to set up device-specific execution, manage buffers, and synchronize tasks with the host.

In [ ]:
%%writefile compute.cpp
#include <sycl/sycl.hpp>
#include <vector>
#include <iostream>

int main(int argc, char **argv) {
    
    const int N = 10;
    std::vector<int> vectorA(N);  // Vector A filled with 1s
    const int p = 4;
    
    {
        auto myQueue = sycl::queue{sycl::gpu_selector_v};
        std::cout << "Running on "
                  << myQueue.get_device().get_info<sycl::info::device::name>()
                  << "\n";

    
        auto bufA = sycl::buffer<int>{ vectorA.data(), vectorA.size()};
        //sycl::buffer<int> bufA {vectorA.data(),vectorA.size() };
        
        myQueue.submit([&](sycl::handler &cgh) {
        
            sycl::accessor accA { bufA, cgh, sycl::write_only, sycl::no_init};
            
            cgh.parallel_for(N, [=](auto i) {
                accA[i] = i * p; 
                });
        
            });
        myQueue.wait();
        
    }

    for (int i = 0; i < N; i++)  // Print output
        std::cout << vectorA[i] << " "; std::cout << "\n";

    return 0;
}


Below we again write a shell script named `run-sycl.sh`, which sets up the environment, compiles the SYCL program using the DPC++ compiler (`icpx`), and runs the compiled executable if the compilation succeeds.

In [ ]:
%%writefile ./run-sycl.sh
#!/bin/bash
source /opt/intel/oneapi/setvars.sh > /dev/null 2>&1
icpx -fsycl compute.cpp
if [ $? -eq 0 ]; then ./a.out; fi

Lastly, the cell below makes the shell script executable and runs it, which will compile and execute the SYCL program on the selected device

In [ ]:
!chmod u+x ./run-sycl.sh &&./run-sycl.sh